# 問題34: 名詞の連接

形態素解析結果から名詞の連接（連続して出現する名詞）を抽出せよ．

In [ ]:
# 問題30で実装した関数を読み込む
import sys
sys.path.append('../..')  # 親ディレクトリをパスに追加

# 形態素解析結果を読み込む関数
def load_mecab_result(file_path):
    """
    MeCabの解析結果ファイルを読み込み、各形態素を辞書のリストとして返す関数
    
    Args:
        file_path (str): MeCab出力ファイルのパス
        
    Returns:
        list: 文のリスト。各文は形態素（辞書）のリスト
    """
    sentences = []
    current_sentence = []
    
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            # EOSは文の区切り
            if line == 'EOS\n':
                if current_sentence:
                    sentences.append(current_sentence)
                    current_sentence = []
                continue
                
            # 空行をスキップ
            if line == '\n':
                continue
                
            # タブで分割して表層形とそれ以外の情報に分ける
            try:
                surface, info = line.split('\t')
                
                # カンマで分割して品詞情報などを取得
                info_items = info.split(',')
                
                # 形態素情報を辞書として格納
                morpheme = {
                    'surface': surface,
                    'base': info_items[6],
                    'pos': info_items[0],
                    'pos1': info_items[1]
                }
                
                current_sentence.append(morpheme)
            except:
                # 不正な形式の行をスキップ
                continue
    
    # 最後の文が追加されていない場合に追加
    if current_sentence:
        sentences.append(current_sentence)
    
    return sentences

# 名詞の連接を抽出する関数
def extract_noun_sequences(sentences):
    """
    形態素解析結果から名詞の連接（連続して出現する名詞）を抽出する関数
    
    Args:
        sentences (list): 文のリスト。各文は形態素（辞書）のリスト
        
    Returns:
        list: 名詞の連接のリスト（各連接は表層形を連結した文字列）
    """
    noun_sequences = []
    
    # 全ての文を処理
    for sentence in sentences:
        current_sequence = []
        
        # 文中の各形態素を調べる
        for morpheme in sentence:
            # 名詞の場合、現在の連接に追加
            if morpheme['pos'] == '名詞':
                current_sequence.append(morpheme['surface'])
            else:
                # 名詞以外の場合、これまでの連接を処理
                if len(current_sequence) >= 2:  # 2つ以上の名詞からなる連接のみを対象
                    noun_sequences.append(''.join(current_sequence))
                current_sequence = []
        
        # 文の最後の連接を処理
        if len(current_sequence) >= 2:
            noun_sequences.append(''.join(current_sequence))
    
    return noun_sequences

In [ ]:
# 形態素解析結果の読み込み
file_path = '../../data/neko.txt.mecab'
sentences = load_mecab_result(file_path)

# 名詞の連接を抽出
noun_sequences = extract_noun_sequences(sentences)

# 結果の表示
print(f"名詞の連接の数: {len(noun_sequences)}")
print("\n最初の20個の名詞の連接:")
for sequence in noun_sequences[:20]:
    print(sequence)

# 連接の長さ（文字数）でソート
sorted_sequences = sorted(noun_sequences, key=len, reverse=True)
print("\n最も長い10個の名詞の連接:")
for sequence in sorted_sequences[:10]:
    print(f"{sequence} ({len(sequence)}文字)")

# 重複を除いた名詞の連接
unique_sequences = set(noun_sequences)
print(f"\n重複を除いた名詞の連接の数: {len(unique_sequences)}")

## 解説

この問題では、形態素解析結果から名詞の連接（連続して出現する名詞）を抽出しました。

### 実装のポイント

1. **連続する名詞の検出**: 各文の中で、品詞が「名詞」である形態素が連続している部分を検出しています。

2. **連接の構築**: 連続する名詞の表層形を連結して、名詞の連接を作成しています。

3. **最小長の設定**: 2つ以上の名詞からなる連接のみを抽出しています。1つの名詞だけでは「連接」とは言えないためです。

4. **文境界の処理**: 文の最後に名詞の連接がある場合も、適切に処理しています。

### 名詞の連接の特徴

名詞の連接は、以下のようなケースで発生します：

1. **複合名詞**: 「東京都庁」「高校野球」など、複数の名詞が組み合わさって1つの概念を表す場合。

2. **名詞の修飾関係**: 「大学教授」「日本文化」など、前の名詞が後ろの名詞を修飾する関係。

3. **並列関係**: 「山川湖」「父母兄弟」など、複数の名詞が並列的に並んでいる場合。

### 結果の考察

抽出された名詞の連接の中には、小説『吾輩は猫である』の特徴を反映したものが多く含まれています。例えば、登場人物や場所に関連する複合名詞、時代背景を示す表現などが見られます。

また、名詞の連接の長さ（構成する名詞の数）は、文の複雑さや文体の特徴を示す指標となります。長い名詞の連接が多い場合、文章は情報密度が高く、簡潔な表現が多いと言えます。